In [1]:
import pandas as pd
import datetime
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
from sqlalchemy import create_engine
#!pip install ipython-sql
%reload_ext sql

In [2]:
date_cols = ["Filing Date", "Issuance Date", "Expiration Date", "Job Start Date"]
permits = pd.read_csv(r'C:\Users\rayde\Downloads\Historical_DOB_Permit_Issuance.csv', low_memory=False, parse_dates=date_cols)
POPULATION = pd.read_csv(r"C:\Users\rayde\Downloads\2010+Census+Population+By+Zipcode+(ZCTA).csv", low_memory=False)

In [5]:
# Check to see dates imported as datetime64

permits[date_cols].dtypes

Filing Date        datetime64[ns]
Issuance Date      datetime64[ns]
Expiration Date    datetime64[ns]
Job Start Date             object
dtype: object

In [6]:
#Job Start Date Column is an object. Need to convert to datetime64
permits['Job Start Date'] = pd.to_datetime(permits['Job Start Date'],errors="coerce")

In [7]:
#Check Columns again
columns = ["Filing Date", "Issuance Date", "Expiration Date", "Job Start Date"]
# columns need to reformat "Filing Date" "Issuance Date" "Expiration Date" "Job Start Date"

permits[date_cols].dtypes

Filing Date        datetime64[ns]
Issuance Date      datetime64[ns]
Expiration Date    datetime64[ns]
Job Start Date     datetime64[ns]
dtype: object

In [25]:
#Check for NaN values in the column
permits.isnull().sum()
#We are missing only 4 number and 4 streets but we are missing the block, lot and postcode, bldg type ... etc 
# of a great deal more than 4 observations. We can probably look this information up. (No need for imputation)

BOROUGH                                   0
BIN                                       0
Number                                    4
Street                                    4
Job #                                     0
Job doc. #                                0
Job Type                                  0
Self_Cert                            688837
Block                                   284
Lot                                     293
Community Board                         614
Postcode                                269
Bldg Type                             24097
Residential                          705895
Special District 1                   981418
Special District 2                  1047758
Work Type                            193057
Permit Status                          3660
Filing Status                             0
Permit Type                               0
Permit Sequence #                         0
Permit Subtype                       426168
Oil Gas                         

In [27]:
#We are missing 21.4% of the data
permits.isnull().sum().sum()/np.product(permits.shape)

0.21403342631666786

In [28]:
engine = create_engine('sqlite:///permitsdb.sql', echo=False)
permits.to_sql('permits', con=engine, if_exists='replace')
POPULATION.to_sql('POPULATION', con=engine, if_exists='replace')

In [29]:
import os
os.environ["DATABASE_URL"]='sqlite:///permitsdb.sql'

In [30]:
%%sql

SELECT * FROM permits LIMIT 20

Done.


index,BOROUGH,BIN,Number,Street,Job #,Job doc. #,Job Type,Self_Cert,Block,Lot,Community Board,Postcode,Bldg Type,Residential,Special District 1,Special District 2,Work Type,Permit Status,Filing Status,Permit Type,Permit Sequence #,Permit Subtype,Oil Gas,Site Fill,Filing Date,Issuance Date,Expiration Date,Job Start Date,Permittee's First Name,Permittee's Last Name,Permittee's Business Name,Permittee's Phone #,Permittee's License Type,Permittee's License #,Act as Superintendent,Permittee's Other Title,HIC License,Site Safety Mgr's First Name,Site Safety Mgr's Last Name,Site Safety Mgr Business Name,Superintendent First & Last Name,Superintendent Business Name,Owner's Business Type,Non-Profit,Owner's Business Name,Owner's First Name,Owner's Last Name,Owner's House #,Owner's House Street Name,Owner’s House City,Owner’s House State,Owner’s House Zip Code,Owner's Phone #,DOBRunDate,Latitude,Longitude,Council District,Census Tract,BBL,NTA
0,BRONX,2118801,2960,WEBSTER AVENUE,201088492,4,NB,None,3274.0,4,207,10458.0,2.0,None,None,None,PL,ISSUED,INITIAL,PL,1,None,None,None,2010-11-05 00:00:00.000000,2010-11-05 00:00:00.000000,2011-11-05 00:00:00.000000,2010-11-05 00:00:00.000000,LAWRENCE,LEVINE,"PAR PLUMBING CO., INC",2129261088,MASTER PLUMBER,161,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2016-01-03T00:00:00,40.867490000000004,-73.883225,11.0,425.0,2032740001.0,Norwood
1,BRONX,2096812,100,DEKRUIF PLACE,200716298,2,A2,None,5141.0,120,209,10475.0,2.0,None,None,None,EQ,ISSUED,RENEWAL,EQ,12,FN,None,NONE,2012-01-30 00:00:00.000000,2012-01-30 00:00:00.000000,2013-01-29 00:00:00.000000,2002-08-08 00:00:00.000000,ANTHONY,RASULO,RIVERBAY CORP,7183203300,GENERAL CONTRACTOR,1962,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2016-01-03T00:00:00,40.875769,-73.82889899999999,12.0,46201.0,2051410120.0,Co-op City
2,BRONX,2008604,1898,HARRISON AVENUE,200974650,2,A2,None,2869.0,87,205,10453.0,2.0,None,None,None,PL,ISSUED,RENEWAL,PL,3,None,None,NONE,2008-02-04 00:00:00.000000,2008-02-04 00:00:00.000000,2009-02-03 00:00:00.000000,2005-08-29 00:00:00.000000,OSCAR,JACKSON,PERFECT PLUMBING & HETING CORP,7185157055,MASTER PLUMBER,594,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2016-01-03T00:00:00,40.852603,-73.911461,14.0,243.0,2028690087.0,University Heights-Morris Heights
3,BRONX,2007652,1998,MORRIS AVENUE,200278118,2,A1,None,2807.0,15,205,10453.0,1.0,None,None,None,PL,ISSUED,INITIAL,PL,1,None,None,NONE,1998-08-31 00:00:00.000000,1998-08-31 00:00:00.000000,1999-08-31 00:00:00.000000,1998-08-31 00:00:00.000000,GERI,KAUUMBA,GOWIE PLUMBING,7188821281,MASTER PLUMBER,1137,Y,None,None,None,None,None,GOWIE PLUMBING,GOWIE PLUMBING,None,None,None,None,None,None,None,None,None,None,None,2016-01-03T00:00:00,40.851661,-73.906937,14.0,241.0,2028070015.0,Mount Hope
4,BRONX,2084155,565,WEST 235 STREET,201119173,2,A2,Y,5794.0,484,208,10463.0,2.0,None,None,None,MH,ISSUED,INITIAL,EW,1,MH,None,NONE,2007-04-30 00:00:00.000000,2007-04-30 00:00:00.000000,2008-01-08 00:00:00.000000,2007-04-30 00:00:00.000000,GARY,ZYSMAN,THE DU-RITE INC,2013877000,GENERAL CONTRACTOR,9872,Y,None,None,None,None,None,THE DU-RITE INC,THE DU-RITE INC,None,None,None,None,None,None,None,None,None,None,None,2016-01-03T00:00:00,40.88572,-73.91027,11.0,297.0,2057940484.0,Spuyten Duyvil-Kingsbridge
5,BRONX,2012264,606,EAST FORDHAM ROAD,200089251,2,A1,None,3078.0,16,206,10458.0,2.0,None,None,None,EQ,ISSUED,INITIAL,EQ,1,FN,None,NONE,1994-04-22 00:00:00.000000,1994-04-22 00:00:00.000000,1995-04-22 00:00:00.000000,1994-04-22 00:00:00.000000,JOSE,VARGAS,VARGAS ASSOCIATES,9149692853,PROFESSIONAL ENGINEER,56795,Y,None,None,None,None,None,VARGAS ASSOCIATES,VARGAS ASSOCIATES,None,None,None,None,None,None,None,None,None,None,None,2016-01-03T00:00:00,40.858323999999996,-73.88483599999999,15.0,389.0,2030780016.0,Belmont
6,BRONX,2103486,730,CONCOURSE VILLAGE WEST,20089676

In [31]:
%%sql
SELECT COUNT("Filing Status") FROM permits
WHERE "Filing Status" = "RENEWAL"

 * sqlite:///permitsdb.sql
Done.


"COUNT(""Filing Status"")"
271030


In [32]:
%%sql 
SELECT BOROUGH, COUNT(*) AS COUNT FROM permits 
WHERE (SELECT ("Expiration Date" - "Issuance Date") AS DATE_DIFF)>=1
GROUP BY BOROUGH
ORDER BY COUNT DESC;

 * sqlite:///permitsdb.sql
Done.


BOROUGH,COUNT
BROOKLYN,367617
MANHATTAN,217507
BRONX,147328


In [33]:
%%sql
SELECT BOROUGH, MAX(COUNT)
FROM (SELECT BOROUGH,COUNT(*) AS COUNT 
FROM permits
WHERE (SELECT ("Expiration Date" - "Issuance Date") AS DATE_DIFF)>=1
GROUP BY BOROUGH);

 * sqlite:///permitsdb.sql
Done.


BOROUGH,MAX(COUNT)
BROOKLYN,367617


In [16]:
%%sql
SELECT MAX/SUM
FROM (SELECT BOROUGH, CAST(MAX(COUNT) AS FLOAT) AS MAX, CAST(SUM(COUNT) AS FLOAT) AS SUM
FROM (SELECT BOROUGH,COUNT(*) AS COUNT 
FROM permits
WHERE (SELECT ("Expiration Date" - "Issuance Date") AS DATE_DIFF)>=1
GROUP BY BOROUGH));

 * sqlite:///permitsdb.sql
Done.


MAX/SUM
0.5018991005553948


In [34]:
%%sql 
SELECT COUNT(*)
FROM permits 
WHERE "Owner's Business Type"="CORPORATION" AND "Filing Status" = "RENEWAL";

 * sqlite:///permitsdb.sql
Done.


COUNT(*)
78144


In [18]:
%%sql
SELECT COUNT(*)
FROM permits 
WHERE "Owner's Business Type"="INDIVIDUAL" AND "Filing Status" = "RENEWAL";

 * sqlite:///permitsdb.sql
Done.


COUNT(*)
79054


In [19]:
%%sql
+6321.0

SELECT "Owner's Business Type", COUNT(*) AS COUNT
FROM permits
WHERE "Filing Status" = "RENEWAL" AND "Owner's Business Type" IN ("CORPORATION",'INDIVIDUAL')
GROUP BY "Owner's Business Type";

 * sqlite:///permitsdb.sql
Done.


Owner's Business Type,COUNT
CORPORATION,78144
INDIVIDUAL,79054


In [20]:
sql_query = '''SELECT "Owner's Business Type", COUNT(*) AS COUNT
FROM permits
WHERE "Filing Status" = "RENEWAL" AND "Owner's Business Type" IN ("CORPORATION",'INDIVIDUAL')
GROUP BY "Owner's Business Type"'''
counts = pd.read_sql_query(sql_query, con='sqlite:///permitsdb.sql')
print(counts)
ratio = counts["COUNT"].loc[0]/counts["COUNT"].loc[1]
print("ratio:",ratio)

  Owner's Business Type  COUNT
0           CORPORATION  78144
1            INDIVIDUAL  79054
ratio: 0.9884888810180383


In [21]:
%%sql 
SELECT Postcode, COUNT(*) AS NUM_PERMITS
FROM permits
WHERE "Filing Date" BETWEEN '2010-01-01 00:00:00.000' AND '2010-12-31 23:59:59.998'
GROUP BY Postcode
ORDER BY COUNT(*) DESC;

 * sqlite:///permitsdb.sql
Done.


Postcode,NUM_PERMITS
11201.0,2179
11215.0,1552
11211.0,1288
11238.0,1211
11220.0,1187
11235.0,1129
11219.0,1127
11217.0,1091
11222.0,990
11223.0,983


In [36]:
%%sql

SELECT Postcode, COUNT(*) AS NUM_PERMITS, POPULATION."2010 Census Population" As POPULATION
FROM permits
LEFT JOIN POPULATION
ON POPULATION."2010 Census Population" = permits.Postcode
WHERE POPULATION."2010 Census Population" >=1000 AND permits."Filing Date" BETWEEN '2010-01-01 00:00:00.000' AND '2010-12-31 23:59:59.998'
GROUP BY Postcode
ORDER BY COUNT(*) DESC
LIMIT 5;

 * sqlite:///permitsdb.sql
Done.


Postcode,NUM_PERMITS,POPULATION
11201.0,2179,11201
10456.0,1282,10456
10460.0,1200,10460
11235.0,1129,11235
10021.0,1088,10021


In [22]:
sql_query = '''SELECT Postcode, COUNT(*) AS NUM_PERMITS, POPULATION."2010 Census Population"
FROM permits
LEFT JOIN POPULATION
ON POPULATION."2010 Census Population" = permits.Postcode
WHERE POPULATION."2010 Census Population" >=1000 AND permits."Filing Date" BETWEEN '2010-01-01 00:00:00.000' AND '2010-12-31 23:59:59.998'
GROUP BY Postcode
ORDER BY COUNT(*) DESC'''
joined = pd.read_sql_query(sql_query, con='sqlite:///permitsdb.sql')
print(joined)

    Postcode  NUM_PERMITS  2010 Census Population
0    11201.0         2179                   11201
1    10456.0         1282                   10456
2    10460.0         1200                   10460
3    11235.0         1129                   11235
4    10021.0         1088                   10021
5    10022.0          971                   10022
6    10459.0          968                   10459
7    10003.0          928                   10003
8    11206.0          924                   11206
9    11204.0          847                   11204
10   10451.0          800                   10451
11   11249.0          786                   11249
12   10467.0          695                   10467
13   10472.0          632                   10472
14   11237.0          628                   11237
15   10461.0          608                   10461
16   10012.0          592                   10012
17   11210.0          575                   11210
18   11236.0          509                   11236


In [25]:
joined['ratio'] = joined["NUM_PERMITS"]/joined['2010 Census Population']

In [33]:
joined['zscore'] = stats.zscore(joined['ratio'])

In [36]:
len(joined[joined['zscore']>=2])

1

In [52]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [80]:
%%sql

SELECT strftime("%Y", "Issuance Date"), COUNT(*) 
FROM permits
GROUP BY strftime("%Y", "Issuance Date")
LIMIT 5;

 * sqlite:///permitsdb.sql
Done.


"strftime(""%Y"", ""Issuance Date"")",COUNT(*)
1989,177
1990,7499
1991,20038
1992,23367
1993,25595


In [135]:
sql_query = '''SELECT strftime("%Y", "Issuance Date") AS "Issuance_Date", COUNT(*) AS NUM_PERMITS
FROM permits
GROUP BY strftime("%Y", "Issuance Date")'''
to_regress = pd.read_sql_query(sql_query, con='sqlite:///permitsdb.sql')
print(to_regress)

   Issuance_Date  NUM_PERMITS
0           1989          177
1           1990         7499
2           1991        20038
3           1992        23367
4           1993        25595
5           1994        26228
6           1995        26327
7           1996        28010
8           1997        30546
9           1998        34787
10          1999        38489
11          2000        41979
12          2001        44362
13          2002        47965
14          2003        52712
15          2004        58087
16          2005        64750
17          2006        68457
18          2007        70783
19          2008        67955
20          2009        60492
21          2010        59756
22          2011        62973
23          2012        65836
24          2013        21405


In [137]:
to_regress['Issuance_Date'] = to_regress['Issuance_Date'].astype(int)

In [142]:
to_regress = to_regress[(to_regress['Issuance_Date']>1989) & (to_regress['Issuance_Date']<2013)]

In [144]:
x = to_regress["Issuance_Date"]
y = to_regress['NUM_PERMITS']
const = sm.add_constant(x) # adding a constant
 
model = sm.OLS(y, x).fit()
predictions = model.predict(x) 
print_model = model.summary()
print(print_model)

                                 OLS Regression Results                                
Dep. Variable:            NUM_PERMITS   R-squared (uncentered):                   0.857
Model:                            OLS   Adj. R-squared (uncentered):              0.850
Method:                 Least Squares   F-statistic:                              131.5
Date:                Tue, 06 Oct 2020   Prob (F-statistic):                    9.48e-11
Time:                        13:49:27   Log-Likelihood:                         -258.36
No. Observations:                  23   AIC:                                      518.7
Df Residuals:                      22   BIC:                                      519.9
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------

In [150]:
%%sql

SELECT "Job Start Date"
FROM permits
LIMIT 5;

 * sqlite:///permitsdb.sql
Done.


Job Start Date
2010-11-05 00:00:00.000000
2002-08-08 00:00:00.000000
2005-08-29 00:00:00.000000
1998-08-31 00:00:00.000000
2007-04-30 00:00:00.000000


In [44]:
%%sql

SELECT strftime('%m', "Job Start Date") AS MonthStart, strftime('%Y', "Job Start Date") AS YearStart, COUNT(*) AS NUM_PERMITS
FROM permits
GROUP BY "Job Start Date";

 * sqlite:///permitsdb.sql
Done.


MonthStart,YearStart,NUM_PERMITS
None,None,13
10,1989,1
10,1989,1
10,1989,1
10,1989,1
10,1989,2
10,1989,6
10,1989,1
10,1989,1
10,1989,1


In [45]:
sql_query = '''SELECT strftime('%m', "Job Start Date") AS MonthStart, strftime('%Y', "Job Start Date") AS YearStart, COUNT(*) AS NUM_PERMITS
FROM permits
GROUP BY "Job Start Date"'''
to_ratio = pd.read_sql_query(sql_query, con='sqlite:///permitsdb.sql')
print(to_ratio)

     MonthStart YearStart  NUM_PERMITS
0          None      None           13
1            10      1989            1
2            10      1989            1
3            10      1989            1
4            10      1989            1
...         ...       ...          ...
6718         11      2206            1
6719         12      2206            2
6720         02      2207            5
6721         06      2207            1
6722         06      2208            1

[6723 rows x 3 columns]


In [46]:
to_ratio = to_ratio[(to_ratio['YearStart'].astype(float)>1989) & (to_ratio['YearStart'].astype(float)<2013)]

In [47]:
months = [1,2,7,8]
to_ratio = to_ratio[to_ratio['MonthStart'].astype(float).isin(months)]

In [48]:
to_ratio_grouped = to_ratio.groupby(['YearStart','MonthStart']).sum()

In [49]:
item_list = []
for i in range(0, len(to_ratio_grouped)): 
        if (i % 4 == 0): 
            item_list.append(i)

In [50]:
x = to_ratio_grouped["NUM_PERMITS"]
results = [(x[i]+x[i+1])/(x[i+2]+x[i+3]) for i in item_list]

In [51]:
to_ratio_grouped


NUM_PERMITS
YearStart MonthStart             
1990      01                  110
          02                  194
          07                 1018
          08                  956
1991      01                 1303
...                           ...
2011      08                 5147
2012      01                 4187
          02                 4219
          07                 4596
          08                 5058

[92 rows x 1 columns]

In [52]:
ratio_df = pd.DataFrame(pd.Series(results, name="Ratio of Summer Start Dates to Winter", index = range(1990, 2013)))
ratio_df

,Ratio of Summer Start Dates to Winter
1990,0.154002
1991,0.643812
1992,1.000725
1993,0.770508
1994,0.802102
1995,0.926290
1996,0.748213
1997,0.751929
1998,0.803644
1999,0.779390


In [53]:
ratio_df.max()

Ratio of Summer Start Dates to Winter    1.000725
dtype: float64

In [54]:
%%sql

SELECT strftime('%m', "Job Start Date") AS MonthStart, CAST((julianday("Expiration Date") - julianday("Issuance Date")) AS integer) AS PermitLength
FROM permits
WHERE permits."Job Start Date" BETWEEN '1990-01-01 00:00:00.000' AND '2012-12-31 23:59:59.998'
LIMIT 20;

 * sqlite:///permitsdb.sql
Done.


MonthStart,PermitLength
11,365
08,365
08,365
08,365
04,253
04,365
06,365
10,365
10,365
07,365


In [55]:
sql_query = '''SELECT strftime('%m', "Job Start Date") AS MonthStart, CAST((julianday("Expiration Date") - julianday("Issuance Date")) AS integer) AS PermitLength
FROM permits
WHERE permits."Job Start Date" BETWEEN "1990-01-01 00:00:00.000" AND "2012-12-31 23:59:59.998"'''
chisq_test = pd.read_sql_query(sql_query, con='sqlite:///permitsdb.sql')

In [56]:
print(chisq_test)
chisq_test['MonthStart'] = chisq_test['MonthStart'].astype(int)
chisq_test.dtypes

        MonthStart  PermitLength
0               11         365.0
1               08         365.0
2               08         365.0
3               08         365.0
4               04         253.0
...            ...           ...
1033844         08         129.0
1033845         08         365.0
1033846         02          12.0
1033847         05         365.0
1033848         09         185.0

[1033849 rows x 2 columns]


MonthStart        int32
PermitLength    float64
dtype: object

In [57]:
chisq_test_grouped = chisq_test.groupby(['MonthStart']).mean()

In [58]:
chisq_test_grouped

,PermitLength
MonthStart,
1,270.299757
2,266.453262
3,268.992811
4,270.919288
5,271.081427
6,268.274203
7,269.485996
8,264.258961
9,260.661893


In [59]:
stats.chisquare(chisq_test_grouped)

Power_divergenceResult(statistic=array([1.11025396]), pvalue=array([0.99991447]))